# Deep Dive Tutorial: Materializing Features

## Learning Objectives

In this tutorial you will learn:
1. How to construct an observation set
2. How features, entities, and observation sets are used together
3. How to preview features
4. How to get historical values
5. How and why to deploy features
6. How to serve and consume deployed features

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalogs

### Load the featurebyte library and connect to the local instance of featurebyte

In [1]:
# library imports
import pandas as pd
import numpy as np

# load the featurebyte SDK
import featurebyte as fb

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

11:33:59 | INFO     | Using configuration file at: /Users/jevonyeoh/.featurebyte/config.yaml
11:33:59 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
11:33:59 | INFO     | (1/4) Starting featurebyte services
Container redis  Running
Container mongo-rs  Running
Container spark-thrift  Running
Container featurebyte-server  Running
Container featurebyte-worker  Running
Container mongo-rs  Waiting
Container mongo-rs  Waiting
Container redis  Waiting
Container redis  Healthy
Container mongo-rs  Healthy
Container mongo-rs  Healthy
11:34:01 | INFO     | (2/4) Creating local spark feature store
11:34:01 | INFO     | (3/4) Import datasets
11:34:07 | INFO     | Dataset grocery already exists, skipping import
11:34:07 | INFO     | Dataset healthcare already exists, skipping import
11:34:07 | INFO     | Dataset creditcard already exists, skipping import
11:34:07 | INFO     | (4/4) Playground environment started successfully. Ready to go

### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.DeepDiveMaterializingFeatures)

Cleaning up existing tutorial catalogs


11:34:08 | INFO     | Catalog activated: deep dive materializing features 20230803:1123


Cleaning catalog: deep dive materializing features 20230803:1123
  1 batch feature tables
  1 batch request tables
  1 historical feature tables
  3 observation tables
Done! |████████████████████████████████████████| 100% in 9.3s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 9.5s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 12.3s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         


11:35:14 | INFO     | Catalog activated: deep dive materializing features 20230803:1135


Building a deep dive catalog for materializing features named [deep dive materializing features 20230803:1135]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.1s (0.16%/s)         
Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.1s
Catalog created and pre-populated with data and features


### Load the tables for this catalog

In [3]:
# get the tables for this catalog
grocery_customer_table = catalog.get_table("GROCERYCUSTOMER")
grocery_items_table = catalog.get_table("INVOICEITEMS")
grocery_invoice_table = catalog.get_table("GROCERYINVOICE")
grocery_product_table = catalog.get_table("GROCERYPRODUCT")

### Create views for the tables in this catalog

In [4]:
# create the views
grocery_customer_view = grocery_customer_table.get_view()
grocery_invoice_view = grocery_invoice_table.get_view()
grocery_items_view = grocery_items_table.get_view()
grocery_product_view = grocery_product_table.get_view()

## How to construct an observation set

Learning Objectives

In this section you will learn:
* the purpose of observation sets
* the relationship between entities, point in time, and observation sets
* how to construct an observation set

### Concept: Materialization

A feature in FeatureByte is defined by the logical plan for its computation. The act of computing the feature is known as Feature Materialization.

The materialization of features is made on demand to fulfill historical requests, whereas for prediction purposes, feature values are generated through a batch process called a "Feature Job". The Feature Job is scheduled based on the defined settings associated with each feature.

### Concept: Observation set

An observation set combines entity key values and historical points-in-time, for which you wish to materialize feature values.

The observation set can be a Pandas DataFrame or an ObservationTable object representing an observation set in the feature store.

### Concept: Point in time

A point-in-time for a feature refers to a specific moment in the past with which the feature's values are associated.

It is a crucial aspect of historical feature serving, which allows machine learning models to make predictions based on historical data. By providing a point-in-time, a feature can be used to train and test models on past data, enabling them to make accurate predictions for similar situations in the future.

An observation set is created as a Pandas DataFrame containing the keys for the primary entity, and points in time. The column name for the primary entity must be its serving name, and the column name for the point in time must be "POINT_IN_TIME".

### Example: Create an observation set based upon events

Some use cases are about events, and require predictions to be triggered when a specified event occurs.

A use case requiring predictions about a grocery customer whenever an invoice event occurs, your observation set may be sampled from historical invoices.

In [5]:
# show the serving name for grocery customer
entity_list = catalog.list_entities()
display(entity_list[entity_list.name == "grocerycustomer"])

,id,name,serving_names,created_at
3,64cb208dc10e38a112bc2024,grocerycustomer,[GROCERYCUSTOMERGUID],2023-08-03T03:35:41.325000


In [6]:
# get a sample of 200 customer IDs and invoice event timestamps from 01-Apr-2022 to 31-Mar-2023
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] <= pd.to_datetime("2023-03-31")
)
observation_set = (
    grocery_invoice_view[filter]
    .sample(200)[["GroceryCustomerGuid", "Timestamp"]]
    .rename(
        {
            "Timestamp": "POINT_IN_TIME",
            "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
        },
        axis=1,
    )
)
display(observation_set)

,GROCERYCUSTOMERGUID,POINT_IN_TIME
0,a900e82a-5742-4929-aaf7-7e79ed5383f2,2022-04-14 20:01:23
1,7a024068-3f99-4114-9d90-3a61f679be51,2022-07-05 16:03:08
2,5b185248-658c-4dbe-bbb7-70d215fb6a05,2022-12-20 07:59:08
3,12c2d702-1b92-4375-8fd4-5b3bd18f7d87,2022-11-13 17:09:40
4,d7316f3d-6ea9-49b6-97f0-3d20ea9d1331,2023-01-31 18:11:55
...,...,...
195,4eb4ee84-ee13-4eec-9c26-61b6eb4ba35b,2022-10-31 09:22:10
196,2b54ef0e-8b02-4f1e-896a-767d23a6162a,2022-09-03 12:17:46
197,3eb57343-4b91-4e06-bed5-c763514c4e64,2022-04-05 18:52:48
198,144a0fe4-2137-43f6-b266-411b9eb7cb31,2023-01-30 14:21:34


### Concept: Observation table

An ObservationTable object is a representation of an observation set in the feature store. Unlike a local Pandas DataFrame, the ObservationTable is part of the catalog and can be shared or reused.

ObservationTable objects can be created from a source table or from a view after subsampling.

### Example: Create an observation table based upon events

In [7]:
# create a large observation table from a view
# observation tables are the recommended workflow for training data

# filter the view to exclude points in time that won't have data for historical windows
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table = observation_set_view.create_observation_table(
    name="10000 customers who were active between 01-Apr-2022 and 31-Mar-2023",
    sample_rows=10000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 21.7s (0.05%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-05 06:51:50,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-05-25 13:01:50,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-05-29 15:35:31,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-06-13 12:02:17,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-06-21 11:36:42,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-07-14 12:20:27,a01ea79c-a9e6-4248-a77e-87cdf2a9bd92
9996,2022-07-23 16:29:02,a01ea79c-a9e6-4248-a77e-87cdf2a9bd92
9997,2022-08-11 12:29:14,a01ea79c-a9e6-4248-a77e-87cdf2a9bd92
9998,2022-08-27 04:39:24,a01ea79c-a9e6-4248-a77e-87cdf2a9bd92


### Example: Create an observation set based upon regularly scheduled batch predictions

Some use cases require predictions to be triggered at regular time periods. Some use cases have conditions for which only a subset of entities require predictions.

A use case requiring monthly predictions for recently active customers may use an observation set containing sample customer IDs combined with predefined timestamps.

In [8]:
# define a function to list a sample of the customers who were active in a given month
def get_recently_active_customers(month_number):
    # filter the invoices by month
    filter = (grocery_invoice_view["Timestamp"].dt.month == month_number) & (
        grocery_invoice_view["Timestamp"].dt.year == 2022
    )
    # get a list of customers who made an invoice in the month
    recently_active_customers = (
        grocery_invoice_view[filter].sample(200)["GroceryCustomerGuid"].unique()
    )
    # get the start of the month
    point_in_time = pd.Timestamp(f"2022-{month_number}-01")
    # get the end of the month
    end_of_month = point_in_time + pd.DateOffset(months=1)
    # get the point in time by subtracting 0.001 second from the end of the month
    point_in_time = end_of_month - pd.Timedelta(seconds=0.001)
    # combine the point in time with the customer IDs
    recently_active_customers = pd.DataFrame(
        {
            "GROCERYCUSTOMERGUID": recently_active_customers,
            "POINT_IN_TIME": point_in_time,
        }
    )
    return recently_active_customers


# create an observation set comprised of up to 200 customers per month who were active in that month in the second half of 2022
observation_set = pd.concat(
    [get_recently_active_customers(month_number) for month_number in range(7, 13)],
    ignore_index=True,
)
display(observation_set)

,GROCERYCUSTOMERGUID,POINT_IN_TIME
0,575ceb64-e6ef-446d-9a38-929e35e4cbef,2022-07-31 23:59:59.999
1,b95f380e-7e7b-4bca-9762-fd9a4fd07419,2022-07-31 23:59:59.999
2,cfd39ed9-3140-4af5-9f72-77881aa6c2a8,2022-07-31 23:59:59.999
3,79b85aee-d548-4e6d-89b0-6969fcce5feb,2022-07-31 23:59:59.999
4,db2d5721-8869-40f7-984c-a94d614fdf69,2022-07-31 23:59:59.999
...,...,...
856,ff38d86f-cd9a-4860-9b0a-eb387bfe0a10,2022-12-31 23:59:59.999
857,5fc2332e-03ac-448d-bf34-f3322cdc295e,2022-12-31 23:59:59.999
858,6132395b-aa85-4fc7-849d-8b8bbd47e1f9,2022-12-31 23:59:59.999
859,c6ef9073-3351-4f54-869a-4c926a479520,2022-12-31 23:59:59.999


## Previewing features

Learning Objectives

In this section you will learn:
* how to preview features
* the limitations of previews

### Example: Preview features

During feature prototyping, new features may not have been saved to the catalog. A data scientist will want to preview sample features to sensibility check their feature declaration.

In [9]:
# create a lookup feature that is the city in which the customer resides
french_state_lookup = grocery_customer_view.City.as_feature("CustomerCity")

# preview materialized values for the unsaved feature
display(french_state_lookup.preview(observation_set.sample(5)))

,GROCERYCUSTOMERGUID,POINT_IN_TIME,CustomerCity
795,a0090791-164f-4089-9d87-e997674b965e,2022-12-31 23:59:59.999,NICE
383,923e9190-d94f-4af9-b909-af0cb368e724,2022-09-30 23:59:59.999,VILLENEUVE-SAINT-GEORGES
31,489b3454-bd45-4d14-a355-500f42bad6c2,2022-07-31 23:59:59.999,PARIS
663,05b77e7e-0a50-4341-b18c-be77efe3b084,2022-11-30 23:59:59.999,MONTAUBAN
280,d8de4748-0f7e-41ee-9356-fb7ca7fdcd20,2022-08-31 23:59:59.999,MANTES-LA-JOLIE


Feature previews are not suited to creating training files or feature serving. Previews have a limitation of 50 rows and do not create an audit trail.

## Create training data

Learning Objectives

In this section you will learn:
* how to design an observation set suitable for training data
* how to get historical values for the target
* how to get historical values for a feature list, and create training data

### Design an Observation Set for Training

Observation Training Design: A training data observation set should typically meet the following criteria:
* be collected from a time period that does not start until after the earliest data availability timestamp plus longest time window in the features
* be collected from a time period that ends before the latest data timestamp less the time window of the target value
* uses points in time that align with the anticipated timing of the use case inference, whether it's based on a regular schedule, triggered by an event, or any other timing mechanism.
* does not have duplicate rows
* has a column containing the primary entity of the use case, using its serving name
* has a column, named "POINT_IN_TIME", containing the points in time
* has for the same entity key points in time that have time intervals greater than the horizon of the target to avoid leakage

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

### Example: Create an observation table for training data

In [10]:
# describe the customer view
display(grocery_customer_view.describe())

,RowID,GroceryCustomerGuid,ValidFrom,Gender,Title,GivenName,MiddleInitial,Surname,StreetAddress,City,State,PostalCode,BrowserUserAgent,DateOfBirth,Latitude,Longitude
dtype,VARCHAR,VARCHAR,TIMESTAMP,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,VARCHAR,DATE,FLOAT,FLOAT
unique,530,500,530,2,4,347,26,352,512,300,27,353,82,495,530,530
%missing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
%empty,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
entropy,6.214608,6.191446,NaN,0.692285,1.146938,5.726251,2.925542,5.749627,6.201803,5.435211,2.49532,5.763347,3.814598,NaN,NaN,NaN
top,0069200d-adf5-490a-acca-14bdf78072a0,0b7196a2-2dab-4218-a234-e193f7bc4470,2019-01-01 07:23:45,male,Mr.,Joanna,A,Saindon,1 cours Jean Jaures,PARIS,Île-de-France,75004,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,NaN,-12.704022,-0.102024
freq,1.0,3.0,1.0,276.0,264.0,5.0,66.0,6.0,2.0,25.0,189.0,5.0,51.0,NaN,1.0,1.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.50512,2.383389
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.108698,7.822694
min,NaN,NaN,2019-01-01T07:23:45.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1937-07-02T00:00:00.000000000,-12.71811,-61.12404


Note that there are 471 unique customers

In [11]:
# describe the invoice view
display(grocery_invoice_view.describe())

,GroceryInvoiceGuid,GroceryCustomerGuid,Timestamp,tz_offset,Amount
dtype,VARCHAR,VARCHAR,TIMESTAMP,VARCHAR,FLOAT
unique,39828,500,39803,4,6668
%missing,0.0,0.0,0.0,0.0,0.0
%empty,0,0,NaN,0,NaN
entropy,6.214608,5.824943,NaN,0.817283,NaN
top,000949fe-1884-40bb-939d-a52df200981f,3019bdbf-667c-4081-acb5-26cd2d559c5e,2022-01-05 11:34:17,+02:00,1.0
freq,1.0,639.0,2.0,22375.0,834.0
mean,NaN,NaN,NaN,NaN,18.355359
std,NaN,NaN,NaN,NaN,22.735611
min,NaN,NaN,2022-01-01T04:17:46.000000000,NaN,0.0


Note that the earliest data timestamp is at the beginning of 2022, and the timestamps end in the present.

In [12]:
# get the customer feature list
customer_feature_list = catalog.get_feature_list("CustomerFeatures")

# display details about the features in the customer feature list
display(customer_feature_list.list_features())

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.2s


,id,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at,is_default
0,64cb208ec10e38a112bc202c,StateMeanLongitude,V230803,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.725000,True
1,64cb208ec10e38a112bc202b,StateMeanLatitude,V230803,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.453000,True
2,64cb208ec10e38a112bc202a,CustomerInventoryMostFrequent_4w,V230803,VARCHAR,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:44.165000,True
3,64cb208ec10e38a112bc2029,CustomerInventoryEntropy_4w,V230803,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:43.525000,True


Note that the longest time window in the features is 4 weeks.

In [13]:
# get the target
next_customer_sales_14d_target = catalog.get_target("next_customer_sales_14d")

# display details about the target feature
display(next_customer_sales_14d_target.info())

{
  'id': '64cb2094c10e38a112bc202f',
  'target_name': 'next_customer_sales_14d',
  'entities': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'deep dive materializing features 20230803:1135'
    }
  ],
  'window': '14d',
  'has_recipe': True,
  'created_at': '2023-08-03T03:35:48.833000',
  'updated_at': None,
  'primary_table': [
    {
      'name': 'GROCERYINVOICE',
      'status': 'PUBLIC_DRAFT',
      'catalog_name': 'deep dive materializing features 20230803:1135'
    }
  ],
  'metadata': {
    'input_columns': {
      'Input0': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Timestamp',
        'semantic': 'event_timestamp'
      },
      'Input1': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Amount',
        'semantic': None
      }
    },
    'derived_columns': {},
    'aggregations': {
      'F0': {
        'name': 'next_customer_sales_14d',
        'column': 'Input1',
        'function': 'sum',
        'keys': [
          'GroceryCustomerGuid'
        ],
        'window': '14d',
        'category': None,
        'filter': False
      }
    },
    'post_aggregation': {
      'name': 'next_customer_sales_14d',
      'inputs': [
        'F0'
      ],
      'transforms': [
        'is_null',
        'conditional'
      ]
    }
  },
  'namespace_description': None,
  'description': None
}

Note that the time window for the target is 14 days

We can conclude that it would be safe for the training data observation set's points in time to commence on 29-Jan-2022 and end 14 days before the present.<br>

We will create an observation set for invoice dates from Feb-22 to Dec-22.

In [14]:
# create a large observation table from a view

# filter to get Feb-22 to Jan-23
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-02-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table_large = observation_set_view.create_observation_table(
    name="1000 customers who were active between 01-Feb-2022 and 31-Jan-2023",
    sample_rows=1000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table_large.to_pandas())

Done! |████████████████████████████████████████| 100% in 21.8s (0.05%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-05-29 15:35:31,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-11-05 19:21:57,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-05-07 14:58:30,abdef773-ab72-43b6-8e77-050804c1c5fc
3,2022-05-17 15:11:30,abdef773-ab72-43b6-8e77-050804c1c5fc
4,2022-07-13 16:22:39,abdef773-ab72-43b6-8e77-050804c1c5fc
...,...,...
995,2022-12-02 17:06:40,bfb599c9-404c-42c1-addf-84b7b1b42ca8
996,2022-12-28 20:09:12,5fb1a274-1d0a-422a-8d48-a6a55ddc22a7
997,2022-12-22 12:38:45,2f1f3abd-6b2c-4222-89d1-f2e83b3ad804
998,2022-06-06 07:31:57,bbaff8e5-44ab-4f61-a4e6-405f274bf429


### Example: Get historical values

In [15]:
# use the get historical features function to get the feature values for the observation set
training_data_features = customer_feature_list.compute_historical_features(observation_set)
display(training_data_features)

Done! |████████████████████████████████████████| 100% in 38.0s (0.03%/s)        
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         


,GROCERYCUSTOMERGUID,POINT_IN_TIME,CustomerInventoryEntropy_4w,CustomerInventoryMostFrequent_4w,StateMeanLatitude,StateMeanLongitude
0,575ceb64-e6ef-446d-9a38-929e35e4cbef,2022-07-31 23:59:59.999,2.257205,Pizza Surgelées,48.906913,4.453320
1,b95f380e-7e7b-4bca-9762-fd9a4fd07419,2022-07-31 23:59:59.999,1.927392,"Colas, Thés glacés et Sodas",48.737227,2.240549
2,cfd39ed9-3140-4af5-9f72-77881aa6c2a8,2022-07-31 23:59:59.999,3.309872,Pains,48.737227,2.240549
3,79b85aee-d548-4e6d-89b0-6969fcce5feb,2022-07-31 23:59:59.999,2.614161,"Colas, Thés glacés et Sodas",47.176003,6.023457
4,db2d5721-8869-40f7-984c-a94d614fdf69,2022-07-31 23:59:59.999,2.153532,Bières et Cidres,48.737227,2.240549
...,...,...,...,...,...,...
856,ff38d86f-cd9a-4860-9b0a-eb387bfe0a10,2022-12-31 23:59:59.999,3.133063,Sirops,45.500198,5.054081
857,5fc2332e-03ac-448d-bf34-f3322cdc295e,2022-12-31 23:59:59.999,3.254689,Fromages,48.739038,2.242254
858,6132395b-aa85-4fc7-849d-8b8bbd47e1f9,2022-12-31 23:59:59.999,2.474379,Chips et Tortillas,48.739038,2.242254
859,c6ef9073-3351-4f54-869a-4c926a479520,2022-12-31 23:59:59.999,2.378475,Pizza Surgelées,43.456104,5.887195


### Example: Get target values

We can materialize the Target values by calling `compute_target`, or `compute_target_table`.

In [16]:
# Materialize the target feature using get historical features
training_data_target_table = next_customer_sales_14d_target.compute_target_table(
    observation_table_large,
    observation_table_name="next_customer_sales_14d_target_table"
)
training_data_target = training_data_target_table.to_pandas()
display(training_data_target)

Done! |████████████████████████████████████████| 100% in 27.9s (0.04%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d
0,2022-07-11 17:21:20,c5c71432-a165-4f38-9fad-48ed116802ce,50.34
1,2022-02-08 01:13:52,38d069c8-794b-4c5a-aaf6-3323f1244752,20.73
2,2022-09-21 15:28:59,5f18f733-ef27-423b-8fb7-6172948c9255,26.17
3,2022-07-23 15:32:29,94127b9f-1366-4bbe-afea-7cd77225da52,78.34
4,2022-09-12 17:43:06,914fac66-de1f-4e9e-8ee0-1e5205984072,0.00
...,...,...,...
995,2022-03-28 10:55:06,6b970153-b284-4a4a-a956-ff66abeb5b69,29.61
996,2022-12-22 17:57:31,a900e82a-5742-4929-aaf7-7e79ed5383f2,0.00
997,2022-08-08 09:19:40,7a1bc5dc-e198-419e-b972-0abbdf8903c1,109.46
998,2022-10-16 09:21:01,550a1960-1ca0-4965-b85f-227d1b68ccd4,136.86


### Concept: Historical feature table

A HistoricalFeatureTable object represents a table in the feature store containing historical feature values from a historical feature request. The historical feature values can also be obtained as a Pandas DataFrame, but using a HistoricalFeatureTable object has some benefits such as handling large tables, storing the data in the feature store for reuse, and offering full lineage of the training and test data.

In [17]:
# the syntax is different when using an observation table to create a historical feature table

# Compute the historical feature table
training_table = customer_feature_list.compute_historical_feature_table(
    training_data_target_table,
    historical_feature_table_name="customer training table on 1000 customers who were active between 01-Feb-2022 and 31-Jan-2023",
)
training_data = training_table.to_pandas()
# display the training data
display(training_data)

Done! |████████████████████████████████████████| 100% in 50.9s (0.02%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d,CustomerInventoryEntropy_4w,CustomerInventoryMostFrequent_4w,StateMeanLatitude,StateMeanLongitude
0,2022-02-01 10:54:42,1e6976c5-7622-4e45-8b28-1d75f7ea7793,6.90,2.080983,"Colas, Thés glacés et Sodas",43.457366,5.834499
1,2022-02-01 11:25:09,33a71f26-f36f-423f-8ddb-a7d674102d3b,81.36,3.289136,Café,48.740263,2.236025
2,2022-02-01 12:17:56,d0e9e8fa-bd2d-415f-a3b3-00c3e2f8e358,21.45,2.318967,Yaourt et Compotes,48.740263,2.236025
3,2022-02-01 13:40:04,506efe54-4f6c-4c30-a11e-48cdb69abbbb,96.09,3.013428,Laits,48.740263,2.236025
4,2022-02-01 17:32:14,f6a783f7-5091-46fa-8ebf-aa13ec868234,88.73,2.913922,"Colas, Thés glacés et Sodas",48.740263,2.236025
...,...,...,...,...,...,...,...
995,2022-12-30 14:17:02,41c1bdf5-b596-4fc4-9570-ecd86a0d9a98,104.50,2.183187,"Colas, Thés glacés et Sodas",47.401700,-1.075038
996,2022-12-30 20:26:43,6f2181ca-3de5-47d6-ad26-704cea2f1ee6,47.13,-0.000000,Sucres et Edulcorants,48.739038,2.242254
997,2022-12-30 20:29:19,575ceb64-e6ef-446d-9a38-929e35e4cbef,72.42,-0.000000,"Colas, Thés glacés et Sodas",46.072548,-0.387129
998,2022-12-31 10:20:47,f79b5a63-7863-471d-8c6e-cc1b48bd385b,46.97,3.588900,Fromages,48.739038,2.242254


## Deploying features

Learning Objectives

In this section you will learn:
* feature readiness
* feature list status
* how to deploy a feature list

### Feature readiness

To help differentiate features that are in the prototype stage and features that are ready for production, a feature version can have one of four readiness levels:

PRODUCTION_READY: ready for deployment in production environments.<br>
PUBLIC_DRAFT: shared for feedback purposes.<br>
DRAFT: in the prototype stage.<br>
DEPRECATED`: not advised for use in either training or prediction.

In [18]:
# view the readiness of the features
catalog.list_features()

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64cb208ec10e38a112bc202c,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.731000
1,64cb208ec10e38a112bc202b,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.462000
2,64cb208ec10e38a112bc202a,CustomerInventoryMostFrequent_4w,VARCHAR,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:44.173000
3,64cb208ec10e38a112bc2029,CustomerInventoryEntropy_4w,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:43.536000


When a feature has been reviewed and is ready for production, its readiness can be upgraded.

In [19]:
# get CustomerInventoryEntropy_4w
customer_inventory_entropy_4w = catalog.get_feature("CustomerInventoryEntropy_4w")

In [20]:
# check feature definition file
customer_inventory_entropy_4w.definition

'# Generated by SDK version: 0.4.1\nfrom bson import ObjectId\nfrom featurebyte import DimensionTable\nfrom featurebyte import FeatureJobSetting\nfrom featurebyte import ItemTable\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("64cb2085c10e38a112bc2022"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n        "tz_offset",\n    ],\n)\n\n# dimension_table name: "GROCERYPRODUCT"\ndimension_table = DimensionTable.get_by_id(ObjectId("64cb208dc10e38a112bc2023"))\ndimension_view = dimension_table.get_view(\n    view_mode="manual", drop_column_names=[], column_cleaning_operations=[]\n)\njoined_view = item_view.join(\n    dimension_view, on="GroceryProductGuid", how="left", rsuffix=""\n)\ngrouped = joined_view.groupby(\n    by_keys=["GroceryCustomerGuid"], category="ProductGroup"\n).aggregate_over(\n    value_column=None,\n    method="count",\n    windows=["4w"],\n    feature_names=["CustomerInventory_4w"],\n    feature_job_setting=FeatureJobSetting(\n        blind_spot="0s", frequency="3600s", time_modulo_frequency="90s"\n    ),\n    skip_fill_na=True,\n)\nfeat = grouped["CustomerInventory_4w"]\nfeat_1 = feat.cd.entropy()\nfeat_1.name = "CustomerInventoryEntropy_4w"\noutput = feat_1\noutput.save(_id=ObjectId("64cb208ec10e38a112bc2029"))\n'

In [21]:
# change the readiness to public
customer_inventory_entropy_4w.update_readiness("PRODUCTION_READY")

# view the readiness of the features
catalog.list_features()

,id,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,64cb208ec10e38a112bc202c,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.731000
1,64cb208ec10e38a112bc202b,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-08-03T03:35:44.462000
2,64cb208ec10e38a112bc202a,CustomerInventoryMostFrequent_4w,VARCHAR,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:44.173000
3,64cb208ec10e38a112bc2029,CustomerInventoryEntropy_4w,FLOAT,PRODUCTION_READY,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-08-03T03:35:43.536000


### Feature list status

Feature lists can be assigned one of five status levels to differentiate between experimental feature lists and those suitable for deployment or already deployed.

- DEPLOYED: Assigned to feature list with at least one deployed version.
- TEMPLATE: For feature lists as reference templates or safe starting points.
- PUBLIC_DRAFT: For feature lists shared for feedback purposes.
- DRAFT: For feature lists in the prototype stage.
- DEPRECATED: For outdated or unnecessary feature lists.

In [22]:
# view the status of the feature lists
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64cb208ec10e38a112bc202d,CustomerFeatures,4,DRAFT,False,0.25,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-03T03:35:44.832000


When a feature list is ready for review, its status can be updated.

In [23]:
# get the CustomerFeatures feature list
customer_feature_list = catalog.get_feature_list("CustomerFeatures")

# update the status to PUBLIC_DRAFT
customer_feature_list.update_status("PUBLIC_DRAFT")

# view the status of the feature lists
display(catalog.list_feature_lists())

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.1s


,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64cb208ec10e38a112bc202d,CustomerFeatures,4,PUBLIC_DRAFT,False,0.25,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-03T03:35:44.832000


### Deploying a feature list

In [24]:
# deploy the customer feature list
deployment = customer_feature_list.deploy(make_production_ready=True)
deployment.enable()

# view the status of the feature lists
display(catalog.list_feature_lists())

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.1s
Done! |████████████████████████████████████████| 100% in 6.3s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 37.3s (0.03%/s)        


,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64cb208ec10e38a112bc202d,CustomerFeatures,4,DEPLOYED,True,1.0,1.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-03T03:35:44.832000


### Why deploy?

When you deploy a feature list, behind the scenes the Feature Store starts regularly pre-calculating and caching feature values. This can significantly reduce the latency of feature serving.

## Serving and consuming features

Learning Objectives

In this section you will learn:
* the point in time used for production serving
* how to create a Python function to consume a feature list
* how to consume a feature list

### Point in time for deployment

The production feature serving API uses the current time as its point in time. To consume the feature list, send only the primary entity via the serving name.

### Automatically create a Python function for consuming the API

You can either use a python template or a shell script where the generated code will use the curl command to send the request.

For the python template, set the language parameter value as 'python'.
For the shell script, set the language parameter value as 'sh'.

In [25]:
# get a python template for consuming the feature serving API
deployment.get_online_serving_code(language="python")

Loading Feature(s) |████████████████████████████████████████| 4/4 [100%] in 0.1s


'from typing import Any, Dict\n\nimport pandas as pd\nimport requests\n\n\ndef request_features(entity_serving_names: Dict[str, Any]) -> pd.DataFrame:\n    """\n    Send POST request to online serving endpoint\n\n    Parameters\n    ----------\n    entity_serving_names: Dict[str, Any]\n        Entity serving name values to used for serving request\n\n    Returns\n    -------\n    pd.DataFrame\n    """\n    response = requests.post(\n        url="http://localhost:8088/deployment/64cb21b3c10e38a112bc203a/online_features",\n        headers={"Content-Type": "application/json", "active-catalog-id": "64cb2072c10e38a112bc201f"},\n        json={"entity_serving_names": entity_serving_names},\n    )\n    assert response.status_code == 200, response.json()\n    return pd.DataFrame.from_dict(response.json()["features"])\n\n\nrequest_features([{"GROCERYCUSTOMERGUID": "0041bdff-4917-42d5-bd6d-5a555ac616c5"}])'

Copy the online serving code that was generated above, paste it into the cell below, then run it

In [26]:
# replace the contents of this Python code cell with the output from to_be_deployed.get_online_serving_code(language="python")

### Concept: Batch request table

A BatchRequestTable object is a representation of a table in the feature store that specifies entity values for batch serving.

In [27]:
# this is a new use case, a daily batch run for customers who were active in the latest 24 hours

# filter the invoice view to get customers who had an invoice in the latest 24 hours
batch_request_timestamp = pd.Timestamp.now(tz="utc")
filter = grocery_invoice_view["Timestamp"] > batch_request_timestamp - pd.to_timedelta(
    24, unit="hour"
)
recently_active_view = grocery_invoice_view[filter].copy()

display(recently_active_view.preview())

,GroceryInvoiceGuid,GroceryCustomerGuid,Timestamp,tz_offset,Amount


In [28]:
# create a batch request table from the filtered view
# note that the table does not contain a prediction point in time
# batch requests use the batch run time as the point in time
batch_request_table = recently_active_view.create_batch_request_table(
    "customer batch request for customers active in the latest 24 hours as at "
    + str(batch_request_timestamp),
    columns=["GroceryCustomerGuid"],
    columns_rename_mapping={"GroceryCustomerGuid": "GROCERYCUSTOMERGUID"},
)

Done! |████████████████████████████████████████| 100% in 18.8s (0.05%/s)        


### Concept: Batch feature table

A BatchFeatureTable object is a representation of a table in the feature store that contains feature values from batch serving. The object includes metadata on the Deployment and the BatchRequestTable used to create it.

In [29]:
# enable the deployment - this is a pre-requisite
if not deployment.enabled:
    deployment.enable()

In [30]:
# request batch features
batch_features = deployment.compute_batch_feature_table(
    batch_request_table=batch_request_table,
    batch_feature_table_name="customer batch feature data for customers active in the latest 24 hours as at "
    + str(batch_request_timestamp),
)

Done! |████████████████████████████████████████| 100% in 27.9s (0.04%/s)        


In [31]:
# display the contents of the batch feature table
display(batch_features.to_pandas())

,GROCERYCUSTOMERGUID,CustomerInventoryEntropy_4w,CustomerInventoryMostFrequent_4w,StateMeanLatitude,StateMeanLongitude


In [32]:
# display the batch feature table metadata
batch_features.info()

{
  'name': 'customer batch feature data for customers active in the latest 24 hours as at 2023-08-03 03:41:32.735455+00:00',
  'created_at': '2023-08-03T03:42:22.658000',
  'updated_at': None,
  'description': None,
  'batch_request_table_name': 'customer batch request for customers active in the latest 24 hours as at 2023-08-03 03:41:32.735455+00:00',
  'deployment_name': 'Deployment with CustomerFeatures_V230803',
  'table_details': {
    'database_name': 'spark_catalog',
    'schema_name': 'playground',
    'table_name': 'BATCH_FEATURE_TABLE_64cb2211bae4fc923ea700e1'
  }
}

### Disable a deployment

In [33]:
# disable the feature list deployment
deployment.disable()

Done! |████████████████████████████████████████| 100% in 27.9s (0.04%/s)        


## Next Steps

Now that you've completed the deep dive materializing features tutorial, you can put your knowledge into practice or learn more:<br>
1. Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" catalogs
2. Learn more about feature governance via the "Quick Start Feature Governance" tutorial
3. Learn about data modeling via the "Deep Dive Data Modeling" tutorial